In [1]:
import numpy as np
import pandas as pd

In [2]:
node = pd.read_table("./node.txt",delimiter=",")
node["osmid"].astype("int64")
node = node[["osmid","y","x"]]
node

,osmid,y,x
0,2.558506e+07,39.901987,116.385578
1,2.558508e+07,39.899003,116.386792
2,2.558509e+07,39.898860,116.381862
3,2.558512e+07,39.900544,116.377850
4,2.558512e+07,39.902940,116.377763
...,...,...,...
37481,1.007778e+10,39.996019,116.222735
37482,1.009133e+10,39.896058,116.599433
37483,1.009133e+10,39.773362,116.173259
37484,1.009133e+10,39.775494,116.169075


In [3]:
node_info = {}
num = 0
for i in range(node.shape[0]):
    # print(round(node["osmid"][i]))
    node_info[round(node["osmid"][i])] = [num,node["y"][i],node["x"][i]]
    num = num+1

In [4]:
node_info[25585055]

[0, 39.9019869, 116.3855782]

In [5]:
edge = pd.read_table("./edge.txt",delimiter=",")
edge = edge[["FID","u","v"]]
# node["osmid"].astype("int64")
edge

/home/ncut/下载/yes/envs/new_pytorch/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,FID,u,v
0,0,2.558506e+07,5.336507e+08
1,1,2.558506e+07,8.835881e+09
2,2,2.558506e+07,3.427913e+08
3,3,2.558508e+07,1.414035e+09
4,4,2.558508e+07,9.378036e+09
...,...,...,...
86311,86311,1.009133e+10,4.076207e+09
86312,86312,1.009133e+10,5.345749e+08
86313,86313,1.009133e+10,9.783252e+09
86314,86314,1.009133e+10,1.009133e+10


### 获得静态路网的全局图  邻接矩阵

In [ ]:
A = np.zeros((37486,37486))
for index,item in edge.iterrows():
    if(node_info[round(item["u"])][0]==node_info[round(item["v"])][0]):
        print(node_info[round(item["u"])][0])
        print(node_info[round(item["v"])][0])
    # A[node_info[round(item["u"])][0]][node_info[round(item["v"])][0]] +=1

### 获得训练集的图，为邻接矩阵添加权重

In [ ]:
# 加载训练数据，未划分长度15
trainlist = np.load("./traj/id_ndoe_dir_traj_from_up15_train.npy",allow_pickle=True)
# print(trainlist.shape)

In [ ]:
for trip in trainlist:
    # print(len(trip))
    for edge in trip:
        # print(edge)  # [67134 u:29164 v:14684    69     1   919]  # edgeid,node_u,node_v,angle,象限,taxiid

        A[edge[1]][edge[2]] += 1

### 获得小于15 和 等于 1 的轨迹图，为邻接矩阵加权重

In [ ]:
# 加载数据
traj_low15 = np.load("./traj/id_ndoe_dir_traj_low15.npy",allow_pickle=True)
traj_1 = np.load("./traj/id_ndoe_dir_traj_1.npy",allow_pickle=True)

In [ ]:
for trip in traj_low15:
    print(len(trip))
    for edge in trip:
        # print(edge)  # [67134 u:29164 v:14684    69     1   919]  # edgeid,node_u,node_v,angle,象限,taxiid
        A[edge[1]][edge[2]] += 1

'''
出现问题，traj_1中的轨迹 可以省略掉
'''
# for trip in traj_1:
#     print(len(trip))
#     for edge in trip:
#         print(edge)  # [67134 u:29164 v:14684    69     1   919]  # edgeid,node_u,node_v,angle,象限,taxiid

#         # adj[edge[1]][edge[2]] += 1
#     break

### 保存adj A

In [ ]:
np.save("./A.npy",A)
A.shape

### 将adj 转换成 edge_index 和 edge_wight

In [2]:
A = np.load("./adj/A.npy",allow_pickle=True)
row, col = np.where(A != 0)
print(len(row),len(col))

85322 85322


In [3]:
def adj_matrix_to_edge_list(adj_matrix, self_loop=False):
    edge_list = []
    edge_weight = []
    row, col = np.where(adj_matrix != 0)
    num = 0
    for i in range(len(row)):
        if not self_loop and row[i] == col[i]:  # 给定了 self_loop=False，则忽略对角线上的元素
            num+=1
            continue
        edge_list.append([row[i], col[i]])
        edge_weight.append(int(adj_matrix[row[i], col[i]]))
    return edge_list, edge_weight,num

In [4]:
edge_list, edge_weight,num = adj_matrix_to_edge_list(A,False)

In [5]:
num

137

In [17]:
# np.save("./adj/edge_index")
edge_index = np.array(edge_list)
edge_index.transpose(1,0)
np.save("./adj/edge_index_noselfloop.npy",edge_index.transpose(1,0))

In [20]:
edge_weight = np.array(edge_weight)
np.save("./adj/edge_weight.npy",edge_weight)
edge_weight.shape

(85185,)